# Tutorial 1: Materials Database and Spectral Densities
## SPINTRONIC Quantum Dynamics Simulation Framework - Phase 5-7

This tutorial demonstrates the extended materials database with 13 2D materials and temperature-dependent phonon coupling.

### Learning Objectives
1. Import and use the pseudomode Python module
2. Explore the 13 available 2D materials
3. Compute spectral densities with temperature dependence
4. Visualize phonon coupling strengths
5. Compare materials for quantum coherence applications

## 1. Setup and Imports

In [ ]:
# Import pseudomode framework
import sys
sys.path.insert(0, '../build')  # Adjust path to your build directory
import pseudomode_py as pm

# Standard scientific libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['figure.figsize'] = (12, 6)
mpl.rcParams['font.size'] = 11

print(f"✓ Pseudomode Framework {pm.__version__}")
print(f"  {pm.__phase__}")
print(f"  CUDA available: {pm.__cuda_available__}")
print(f"  Materials in database: {pm.__n_materials__}")

## 2. Explore Available Materials

In [ ]:
# List all materials
materials = pm.list_materials()
print(f"Total materials: {len(materials)}\n")

# Display material properties in a table
print(f"{'Material':<20} {'Band Gap (eV)':<15} {'Mass (m_e)':<15} {'Lattice (Å)':<15}")
print("-" * 65)

for mat in materials:
    props = pm.material_info(mat)
    print(f"{mat:<20} {props['bandgap']:<15.2f} {props['mass']:<15.2f} {props['lattice_const']:<15.2f}")

## 3. Compute Spectral Density for MoS₂

In [ ]:
# Energy grid (in eV)
omega = np.linspace(0.001, 0.15, 500)  # 1 to 150 meV

# Compute spectral density at room temperature
J_MoS2 = pm.spectral_density(omega.tolist(), "MoS2", 300.0)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(omega * 1000, J_MoS2, linewidth=2, label='MoS₂ at 300 K')
plt.xlabel('Energy ω (meV)', fontsize=12)
plt.ylabel('Spectral Density J(ω)', fontsize=12)
plt.title('Phonon Spectral Density of MoS₂ Monolayer', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"Peak spectral density: {max(J_MoS2):.6f}")
print(f"Peak location: {omega[np.argmax(J_MoS2)] * 1000:.2f} meV")

## 4. Temperature Dependence

In [ ]:
# Temperature range
temperatures = [77, 150, 300, 450]
colors = plt.cm.coolwarm(np.linspace(0, 1, len(temperatures)))

plt.figure(figsize=(12, 6))

for T, color in zip(temperatures, colors):
    J = pm.spectral_density(omega.tolist(), "MoS2", float(T))
    plt.plot(omega * 1000, J, linewidth=2, color=color, label=f'{T} K')

plt.xlabel('Energy ω (meV)', fontsize=12)
plt.ylabel('Spectral Density J(ω)', fontsize=12)
plt.title('Temperature-Dependent Spectral Density of MoS₂', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(title='Temperature', fontsize=11)
plt.tight_layout()
plt.show()

print("Observation: Spectral density increases with temperature due to enhanced phonon occupation.")

## 5. Materials Comparison

In [ ]:
# Compare TMDs and other 2D materials
comparison_materials = ['MoS2', 'WSe2', 'graphene', 'hBN', 'GaN']
colors = plt.cm.tab10(np.linspace(0, 1, len(comparison_materials)))

plt.figure(figsize=(14, 7))

for mat, color in zip(comparison_materials, colors):
    J = pm.spectral_density(omega.tolist(), mat, 300.0)
    plt.plot(omega * 1000, J, linewidth=2, color=color, label=mat, alpha=0.8)

plt.xlabel('Energy ω (meV)', fontsize=12)
plt.ylabel('Spectral Density J(ω)', fontsize=12)
plt.title('Comparison of 2D Material Spectral Densities at 300 K', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11, loc='upper right')
plt.xlim(0, 100)
plt.tight_layout()
plt.show()

print("\nKey observations:")
print("- TMDs (MoS₂, WSe₂): Strong optical phonon peaks around 30-50 meV")
print("- Graphene: Broad acoustic/flexural coupling, minimal optical coupling")
print("- hBN: Wide bandgap insulator with weak coupling")
print("- GaN: Strong acoustic coupling suitable for piezoelectric applications")

## 6. Phonon Components Analysis

In [ ]:
# Decompose spectral density into components for MoS2
omega_fine = np.linspace(0.001, 0.15, 1000)

# Get individual components
J_acoustic = pm.SpectralDensity2D.acoustic(omega_fine.tolist(), 0.01, 0.04, 1.5)
J_flexural = pm.SpectralDensity2D.flexural(omega_fine.tolist(), 0.005, 0.02, 0.3, 2.0)
J_optical_1 = pm.SpectralDensity2D.lorentzian_peak(omega_fine.tolist(), 0.048, 0.002, 0.001)
J_optical_2 = pm.SpectralDensity2D.lorentzian_peak(omega_fine.tolist(), 0.050, 0.0015, 0.0008)

plt.figure(figsize=(14, 7))

plt.plot(omega_fine * 1000, J_acoustic, linewidth=2, label='Acoustic (LA/TA)', alpha=0.8)
plt.plot(omega_fine * 1000, J_flexural, linewidth=2, label='Flexural (ZA)', alpha=0.8)
plt.plot(omega_fine * 1000, J_optical_1, linewidth=2, label="Optical (A₁')", alpha=0.8)
plt.plot(omega_fine * 1000, J_optical_2, linewidth=2, label="Optical (E')", alpha=0.8)

J_total = np.array(J_acoustic) + np.array(J_flexural) + np.array(J_optical_1) + np.array(J_optical_2)
plt.plot(omega_fine * 1000, J_total, 'k--', linewidth=3, label='Total', alpha=0.7)

plt.xlabel('Energy ω (meV)', fontsize=12)
plt.ylabel('Spectral Density J(ω)', fontsize=12)
plt.title('MoS₂ Phonon Components', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xlim(0, 80)
plt.tight_layout()
plt.show()

## 7. Material Selection for Quantum Coherence

Estimate decoherence strength by integrating spectral density.

In [ ]:
# Compute integrated spectral density (proxy for decoherence rate)
omega_grid = np.linspace(0.001, 0.2, 300)
all_materials = pm.list_materials()

coherence_metric = {}
for mat in all_materials:
    try:
        J = pm.spectral_density(omega_grid.tolist(), mat, 300.0)
        integrated = np.trapz(J, omega_grid)
        coherence_metric[mat] = integrated
    except:
        coherence_metric[mat] = np.nan

# Sort by decoherence strength (lower is better for coherence)
sorted_materials = sorted(coherence_metric.items(), key=lambda x: x[1] if not np.isnan(x[1]) else 1e10)

plt.figure(figsize=(12, 6))
materials_names = [m[0] for m in sorted_materials]
metrics = [m[1] for m in sorted_materials]

colors = ['green' if m < 0.01 else 'orange' if m < 0.02 else 'red' for m in metrics]
plt.barh(materials_names, metrics, color=colors, alpha=0.7, edgecolor='black')
plt.xlabel('Integrated Spectral Density (arb. units)', fontsize=12)
plt.ylabel('Material', fontsize=12)
plt.title('Material Ranking for Quantum Coherence (Lower = Better)', fontsize=14, fontweight='bold')
plt.grid(True, axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 3 materials for long coherence:")
for i, (mat, metric) in enumerate(sorted_materials[:3], 1):
    print(f"{i}. {mat}: {metric:.6f}")

## 8. Summary and Next Steps

### What We Learned
- 13 2D materials available in the database
- Temperature-dependent spectral densities
- Phonon coupling decomposition (acoustic, flexural, optical)
- Materials comparison for quantum applications

### Next Tutorials
- **Tutorial 2**: Pseudomode fitting and correlation functions
- **Tutorial 3**: Quantum dynamics and coherence time extraction
- **Tutorial 4**: Custom materials and parameter optimization

### Further Reading
- Framework documentation: `/workspace/docs/`
- API reference: Generated Doxygen HTML
- Material database: `src/materials_database.cpp`